In [18]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
import websockets
import json
import datetime
from IPython.display import Audio, display

async def send_audio():
    uri = "wss://athletes-anymore-fwd-functions.trycloudflare.com/ws"

    async with websockets.connect(
        uri,
        ping_interval=None,
        max_size=10 * 1024 * 1024
    ) as websocket:
        # 1. 메타데이터 전송
        meta = {
            "user_id": "test_user_2",
            "timestamp": datetime.datetime.utcnow().isoformat()
        }
        await websocket.send(json.dumps(meta))
        print("✅ 메타데이터 전송 완료")

        # 2. 오디오 전송
        with open("test.wav", "rb") as f:
            audio_data = f.read()
        await websocket.send(audio_data)
        print("✅ 오디오 전송 완료")

        # 3. 첫 번째 응답 받기 (JSON or WAV)
        response = await websocket.recv()

        # 바이너리인지 문자열인지 체크
        if isinstance(response, bytes):
            print("📥 바로 오디오 바이너리 수신")
            pcm_data = response
        else:
            try:
                response_json = json.loads(response)
                print("✅ JSON 응답:", response_json)
                # 그 다음 응답은 바이너리 오디오일 것
                pcm_data = await websocket.recv()
            except json.JSONDecodeError:
                print("❌ 예상치 못한 응답 형식:", response)
                return

        # 4. 저장 및 재생
        output_path = "received_from_server.wav"
        with open(output_path, "wb") as f:
            f.write(pcm_data)
        print(f"🎧 저장 완료: {output_path}")

        display(Audio(output_path))

# 실행
await send_audio()


✅ 메타데이터 전송 완료
✅ 오디오 전송 완료
✅ JSON 응답: {'user_id': 'test_user_2', 'sequence': 0, 'timestamp': '2025-07-15T12:27:25.161345', 'answer': '경기도 화성시 우정읍에 위치한 공룡알 화석산지는 중생대 백악기 공룡알 화석이 대량으로 발견된 곳으로, 천연기념물 제414호로 지정되어 보호받고 있습니다.'}
🎧 저장 완료: received_from_server.wav


In [19]:
# ✅ 1. 전체 세션 목록 요청 클라이언트

import requests

def get_all_sessions():
    uri = "https://athletes-anymore-fwd-functions.trycloudflare.com/sessions"
    response = requests.get(uri)

    if response.ok:
        data = response.json()
        print("🟢 세션 수:", data["active_sessions_count"])
        print("👤 사용자 목록:", data["user_ids"])
    else:
        print("❌ 요청 실패:", response.text)

get_all_sessions()



🟢 세션 수: 2
👤 사용자 목록: ['test_user_1', 'test_user_2']


In [22]:
# ✅ 2. 특정 사용자 히스토리 요청 클라이언트

import requests

def get_user_history(user_id):
    uri = f"https://athletes-anymore-fwd-functions.trycloudflare.com/sessions?user_id={user_id}"
    response = requests.get(uri)

    if response.ok:
        data = response.json()
        print(f"📜 {user_id}의 채팅 기록:")
        for msg in data["chat_history"]:
            print(f"[{msg['role'].upper()}] {msg['content']}\n")
    else:
        print("❌ 요청 실패:", response.text)

get_user_history("test_user_1")


📜 test_user_1의 채팅 기록:
[USER] [text: "\' 우학리 공룡알에 대해서 설명해줘.\' 라는 발화에 대해 핵심만 간결하게 한국어로 답변해줘."
]

[MODEL] [text: "화성시 송산면 우음도에 위치한 공룡알 화석산지는 중생대 백악기(약 8300만~8500만 년 전)에 형성된 것으로 추정되며,  갯벌에서 다양한 종류의 공룡알 화석이 대규모로 발견되어 천연기념물 제414호로 지정되었습니다.\n"
]

[USER] [text: "\' 이전에 질문한 내용이 뭐지?\' 라는 발화에 대해 핵심만 간결하게 한국어로 답변해줘."
]

[MODEL] [text: "우학리 공룡알에 대한 설명을 요청하셨습니다.\n"
]

